In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
from sklearn.linear_model import LinearRegression
from math import sqrt, log
from sklearn import linear_model

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 
              'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [4]:
sales = pd.read_csv("~/Desktop/ML_Washington/WashingtonML/Regression/Week2/kc_house_data.csv", dtype = dtype_dict)

In [7]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

In [9]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [10]:
print(model_all.intercept_)
print(model_all.coef_)

-218136.21403514117
[    0.             0.             0.           134.43931396
     0.             0.             0.             0.
     0.             0.         24750.00458561     0.
 61749.10309071     0.             0.            -0.
     0.        ]


In [14]:
df = pd.DataFrame(model_all.coef_,all_features, columns = ['coefficients'])

In [15]:
df

,coefficients
bedrooms,0.000000
bedrooms_square,0.000000
bathrooms,0.000000
sqft_living,134.439314
sqft_living_sqrt,0.000000
sqft_lot,0.000000
sqft_lot_sqrt,0.000000
floors,0.000000
floors_square,0.000000
waterfront,0.000000


In [16]:
testing = pd.read_csv("~/Desktop/ML_Washington/WashingtonML/Regression/Week4/wk3_kc_house_test_data.csv", dtype = dtype_dict)
training = pd.read_csv("~/Desktop/ML_Washington/WashingtonML/Regression/Week4/wk3_kc_house_train_data.csv", dtype = dtype_dict)
validation = pd.read_csv("~/Desktop/ML_Washington/WashingtonML/Regression/Week4/wk3_kc_house_valid_data.csv", dtype = dtype_dict)


In [17]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [18]:
l1_penalty = np.logspace(1, 7, num=13)

In [19]:
RSS_valid = []
for val in l1_penalty:
    model = linear_model.Lasso(alpha=val, normalize=True)
    model.fit(training[all_features], training['price'])
    
    predictions_valid = model.predict(validation[all_features])
    residuals = predictions_valid - validation['price']
    RSS = sum(value**2 for value in residuals)
    RSS_valid.append(RSS)

In [20]:
RSS_valid

[398213327300134.44,
 399041900253348.4,
 429791604072558.2,
 463739831045119.4,
 645898733633803.2,
 1222506859427156.8,
 1222506859427156.8,
 1222506859427156.8,
 1222506859427156.8,
 1222506859427156.8,
 1222506859427156.8,
 1222506859427156.8,
 1222506859427156.8]

In [21]:
min(RSS_valid)

398213327300134.44

In [22]:
l1_penalty

array([1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07])

In [23]:
model = linear_model.Lasso(alpha=10, normalize=True)
model.fit(training[all_features], training['price'])

predictions_testing = model.predict(testing[all_features])
residuals = predictions_testing - testing['price']
RSS = sum(value**2 for value in residuals)
RSS

98467402552698.92

In [24]:
print(model.intercept_)
print(model.coef_)

6630155.668628358
[-1.61445628e+04  3.73245384e+02  5.08412433e+04  6.17853560e+02
 -4.44113549e+04  7.85623065e-01 -7.01194765e+02 -0.00000000e+00
  5.01420046e+03  6.19488752e+05  3.80418557e+04  2.49987718e+04
  1.28716235e+05  0.00000000e+00  0.00000000e+00 -3.29383118e+03
  1.00573209e+01]


In [25]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [26]:
max_nonzeros = 7

In [27]:
l1_penalty_high = np.logspace(1, 4, num=20)

In [28]:
l1_penalty_high

array([   10.        ,    14.38449888,    20.69138081,    29.76351442,
          42.81332399,    61.58482111,    88.58667904,   127.42749857,
         183.29807108,   263.66508987,   379.26901907,   545.55947812,
         784.75997035,  1128.83789168,  1623.77673919,  2335.72146909,
        3359.81828628,  4832.93023857,  6951.92796178, 10000.        ])

In [34]:
nonzeros_count = []
l1_penalty_min = 0
l1_penalty_max = 100000
for value in l1_penalty_high:
    model = linear_model.Lasso(alpha=value, normalize=True)
    model.fit(training[all_features], training['price'])
    nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzeros_count.append(nonzeros)
    
    if nonzeros > max_nonzeros:
        if value > l1_penalty_min:
            l1_penalty_min = value
    
    if nonzeros < max_nonzeros:        
        if value < l1_penalty_max:
            print("max")
            l1_penalty_max = value
print(l1_penalty_min, l1_penalty_max)

max
127.42749857031335 263.6650898730358


In [30]:
nonzeros_count

[15, 15, 15, 15, 13, 12, 11, 10, 7, 6, 6, 6, 5, 3, 3, 2, 1, 1, 1, 1]

In [35]:
l1_penalty_narrower = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [36]:
l1_penalty_narrower

array([127.42749857, 134.59789811, 141.76829765, 148.9386972 ,
       156.10909674, 163.27949628, 170.44989582, 177.62029537,
       184.79069491, 191.96109445, 199.13149399, 206.30189354,
       213.47229308, 220.64269262, 227.81309216, 234.9834917 ,
       242.15389125, 249.32429079, 256.49469033, 263.66508987])

In [54]:
RSS_valid_narrower = []
RSS_valid_nonzeros = []
for val in l1_penalty_narrower:
    model = linear_model.Lasso(alpha=val, normalize=True)
    model.fit(training[all_features], training['price'])
    nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    
    predictions_valid = model.predict(validation[all_features])
    residuals = predictions_valid - validation['price']
    RSS = sum(value**2 for value in residuals)
    RSS_valid_narrower.append(RSS)
    
    if nonzeros == 7:
        RSS_valid_nonzeros.append(RSS)
    
    if RSS - 440037365263316.9 == 0:
        df_coefficients = pd.DataFrame(model.coef_,all_features, columns = ['coefficients'])
        coefficients = model.coef_
        intercept = model.intercept_
    
    print("For l1_penalty = {} and RSS = {} and the nonzeros count = {}".format(val,RSS,nonzeros))
print ("\n")
print (df_coefficients)
print("\n")
print(coefficients)
print(intercept)

For l1_penalty = 127.42749857031335 and RSS = 435374677102678.94 and the nonzeros count = 10
For l1_penalty = 134.5978981125619 and RSS = 437009229124473.4 and the nonzeros count = 10
For l1_penalty = 141.76829765481045 and RSS = 438236128386911.44 and the nonzeros count = 8
For l1_penalty = 148.938697197059 and RSS = 439158937799659.8 and the nonzeros count = 8
For l1_penalty = 156.10909673930755 and RSS = 440037365263316.9 and the nonzeros count = 7
For l1_penalty = 163.2794962815561 and RSS = 440777489641605.2 and the nonzeros count = 7
For l1_penalty = 170.44989582380464 and RSS = 441566698090138.8 and the nonzeros count = 7
For l1_penalty = 177.6202953660532 and RSS = 442406413188665.7 and the nonzeros count = 7
For l1_penalty = 184.79069490830176 and RSS = 443296716874312.9 and the nonzeros count = 7
For l1_penalty = 191.96109445055032 and RSS = 444239780526140.56 and the nonzeros count = 7
For l1_penalty = 199.13149399279888 and RSS = 445230739842614.06 and the nonzeros count = 

In [39]:
min(RSS_valid_narrower)

435374677102678.94

In [42]:
min(RSS_valid_nonzeros)

440037365263316.9